In [ ]:
import os
import glob
from google.colab import drive

import torch
import torch.nn as nn
import torchvision.models as models
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import torch.optim as optim
from torch.nn import CrossEntropyLoss
from tqdm import tqdm

import cv2
import numpy as np
from PIL import Image
from collections import defaultdict
from sklearn.model_selection import train_test_split


In [ ]:
# Mount Google Drive, if not already mounted

if not os.path.ismount("/content/drive"):
    drive.mount("/content/drive")
else:
    print("Google Drive is already mounted.")

Google Drive is already mounted.


## Download and Extract Dataset

**IMPORTANT**:

Only run the below cells if you want to download the dataset.

If dataset is already downloaded do not run the below cells.

[Dataset Link](https://www.kaggle.com/datasets/karandeep98/real-life-violence-and-nonviolence-data)

[Dataset Link 2](https://www.kaggle.com/datasets/mohamedmustafa/real-life-violence-situations-dataset)

In [ ]:
# Change Working Directory
%cd /content/drive/MyDrive/MSAI/DeepLearning/FinalProject

/content/drive/MyDrive/MSAI/DeepLearning/FinalProject


In [ ]:
DOWNLOAD_DATASET = False
if DOWNLOAD_DATASET:
  from google.colab import files
  files.upload()

In [ ]:
# Make directory to download kaggle dataset

# !mkdir -p ~/.kaggle
# !cp kaggle.json ~/.kaggle/
# !chmod 600 ~/.kaggle/kaggle.json

In [ ]:
# Download

# !kaggle datasets download -d karandeep98/real-life-violence-and-nonviolence-data

Dataset URL: https://www.kaggle.com/datasets/karandeep98/real-life-violence-and-nonviolence-data
License(s): unknown
User cancelled operation


In [ ]:
#Download only if the the data is not already present

if DOWNLOAD_DATASET:
  import zipfile
  import os

  dataset_zip = "real-life-violence-and-nonviolence-data.zip"
  extract_path = "real-life-violence-and-nonviolence-data"

  with zipfile.ZipFile(dataset_zip, 'r') as zip_ref:
      zip_ref.extractall(extract_path)


## Dataset Pre-Processing

There are two data folders

- violence
- non-violence

###### **Note:**
We donot have videos as mp4 but frames as jpg images so for an image all frames are extracted.
Each folder contains the frames of videos, each video has specific id like, V1000.mp4_frame1.jpg, V1000.mp4_frame2.jpg, V1000.mp4_frame3.jpg so on ...




In [ ]:
# Change working directory to FinalProject Folder

%cd /content/drive/MyDrive/violence_detection

In [ ]:
violence_dir = "real-life-violence-and-nonviolence-data/violence_dataset/violence"
non_violence_dir = "real-life-violence-and-nonviolence-data/violence_dataset/non_violence"

In [ ]:
data_len = len(os.listdir(violence_dir))
data_len

5832

In [ ]:
# split dataset function (helping-source : Hands on Macine Learning second edition)
def split_dataset(samples, test_size=0.2, seed=42):
    train_samples, test_samples = train_test_split(samples, test_size=test_size, random_state=seed)
    return train_samples, test_samples

# transform image
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

class VideoDataset(Dataset):
    def __init__(self, samples, num_frames=16, transform=None, _3DCNN = False):
        self.samples = samples
        self.num_frames = num_frames
        self.transform = transform

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, index):
        """
            Dunder method to get item (video, label) pair
        """
        frame_paths, label = self.samples[index]
        selected_frames = frame_paths[:self.num_frames]
        if len(selected_frames) < self.num_frames:
            selected_frames += [selected_frames[-1]] * (self.num_frames - len(selected_frames))

        frames = []
        for frame_path in selected_frames:
            img = Image.open(frame_path).convert("RGB") # convert image to RGB
            if self.transform:
                img = self.transform(img) # apply transform
            frames.append(img)

        video_tensor = torch.stack(frames)  # (T, C, H, W)

        if _3DCNN:
          # Convert to 3D tensor for 3DCNN, shape: [B, C=3, T, H, W]
          video_tensor = video_tensor.permute(0, 2, 1, 3, 4)  # From [B, T, C, H, W] to [B, C, T, H, W]
        return video_tensor, label


In [ ]:
def build_samples(root_dir):
    """
      Build samples
      video: list of frames appended as numeric arrays
      label: numeric representation (0: violence, 1: no-violence)
    """
    samples = []
    for class_name in ["violence", "non_violence"]:
        label = 0 if class_name == "violence" else 1
        class_path = os.path.join(root_dir, class_name)

        # skip if directory not found
        if not os.path.exists(class_path):
            print(f"Directory does not exist: {class_path}")
            continue

        videos = defaultdict(list)
        for file in os.listdir(class_path):
            if file.endswith(".jpg"):
                video_id = "_".join(file.split("_")[:-1])  # V1000.mp4
                videos[video_id].append(os.path.join(class_path, file))

        for video_id, frame_list in videos.items():
            frame_list.sort()
            samples.append((frame_list, label))

    return samples


In [ ]:

root_dir = "real-life-violence-and-nonviolence-data/violence_dataset"

# Build all samples first
all_samples = build_samples(root_dir)

# Split into train (80%) and temp (20%)
train_samples, temp_samples = train_test_split(all_samples, test_size=0.2, random_state=42, stratify=[label for _, label in all_samples])

# Split temp set into validation (10%) and test (10%)
val_samples, test_samples = train_test_split(temp_samples, test_size=0.5, random_state=42, stratify=[label for _, label in temp_samples])

# 3DCNN requires shape [B, C=3, T, H, W]
if model == "3DCNN":
  _3DCNN = True

train_dataset = VideoDataset(train_samples, num_frames=16, transform=transform, _3DCNN = _3DCNN)
val_dataset   = VideoDataset(val_samples, num_frames=16, transform=transform, _3DCNN = _3DCNN)
test_dataset  = VideoDataset(test_samples, num_frames=16, transform=transform, _3DCNN = _3DCNN)

train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)
val_loader   = DataLoader(val_dataset, batch_size=4, shuffle=False)
test_loader  = DataLoader(test_dataset, batch_size=4, shuffle=False)


## Models

In [ ]:
class CNNVideoClassifier(nn.Module):
    def __init__(self, num_classes=2, pretrained=True, freeze_cnn=True):
        super(CNNVideoClassifier, self).__init__()

        # Load pretrained ResNet18
        resnet = models.resnet18(pretrained=pretrained)
        self.cnn = nn.Sequential(*list(resnet.children())[:-1])  # Remove FC layer
        self.feature_dim = 512  # Final feature size from ResNet18

        if freeze_cnn:
            for param in self.cnn.parameters():
                param.requires_grad = False

        # Classification head
        self.classifier = nn.Sequential(
            nn.Linear(self.feature_dim, 128),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(128, num_classes)
        )

    def forward(self, x):  # x: [B, T, C, H, W]
        B, T, C, H, W = x.shape
        x = x.view(B * T, C, H, W)  # Merge batch and time

        with torch.no_grad():  # CNN frozen
            features = self.cnn(x)  # [B*T, 512, 1, 1]
        features = features.view(B, T, self.feature_dim)  # [B, T, 512]

        video_features = features.mean(dim=1)  # Average over frames (T) -> [B, 512]

        out = self.classifier(video_features)  # [B, num_classes]
        return out

In [ ]:
class CNNLSTMModel(nn.Module):
    def __init__(self, hidden_dim=256, lstm_layers=1, num_classes=2):
        super(CNNLSTMModel, self).__init__()

        # Pretrained CNN (ResNet18)
        resnet = models.resnet18(pretrained=True)
        self.cnn = nn.Sequential(*list(resnet.children())[:-1])  # Remove FC layer
        self.feature_dim = 512  # ResNet18 final feature size

        # Freeze CNN (optional)
        for param in self.cnn.parameters():
            param.requires_grad = False

        # LSTM
        self.lstm = nn.LSTM(input_size=self.feature_dim, hidden_size=hidden_dim,
                            num_layers=lstm_layers, batch_first=True)

        # Classifier
        self.classifier = nn.Linear(hidden_dim, num_classes)

    def forward(self, x):  # x: [B, T, C, H, W]
        B, T, C, H, W = x.shape
        x = x.view(B * T, C, H, W)

        with torch.no_grad():  # CNN is frozen
            features = self.cnn(x)  # Output: (B*T, 512, 1, 1)

        features = features.view(B, T, self.feature_dim)  # Reshape to (B, T, 512)
        lstm_out, _ = self.lstm(features)  # (B, T, hidden_dim)
        final_output = lstm_out[:, -1, :]  # Take last timestep
        out = self.classifier(final_output)  # (B, num_classes)
        return out


In [ ]:
class C3DModel(nn.Module):
    def __init__(self, num_classes=2):
        super(C3DModel, self).__init__()

        self.features = nn.Sequential(
            nn.Conv3d(3, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool3d(kernel_size=(1, 2, 2), stride=(1, 2, 2)),  # Time preserved

            nn.Conv3d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool3d(kernel_size=(2, 2, 2), stride=(2, 2, 2)),

            nn.Conv3d(128, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv3d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool3d(kernel_size=(2, 2, 2), stride=(2, 2, 2)),

            nn.Conv3d(256, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv3d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.AdaptiveMaxPool3d((1, 1, 1))  # Output: (B, 512, 1, 1, 1)
        )

        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(512, 128),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(128, num_classes)
        )

    def forward(self, x):  # x: [B, C=3, T, H, W]
        x = self.features(x)  # -> [B, 512, 1, 1, 1]
        out = self.classifier(x)  # -> [B, num_classes]
        return out

# Training

In [ ]:
# Configuration
EPOCHS = 50
PATIENCE = 3
USE_PRETRAINED_WEIGHTS = False  #Set to True to load and continue from saved weights
output_dir = "models/"
model_path = os.path.join(output_dir, "best_model_cnn.pth")
os.makedirs(output_dir, exist_ok=True)

# Device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Model setup
# model = CNNLSTMModel().to(device)
model = CNNVideoClassifier().to(device)

if USE_PRETRAINED_WEIGHTS and os.path.exists(model_path):
    model.load_state_dict(torch.load(model_path))
    print(f"Loaded pre-trained weights from: {model_path}")
else:
    print(f"🚀 Starting training from scratch.")

# Loss and optimizer
criterion = CrossEntropyLoss() # Binary Cross Entropy is better
optimizer = optim.Adam(model.parameters(), lr=1e-4)

# Early stopping variables
best_val_acc = 0.0
epochs_no_improve = 0

for epoch in range(EPOCHS):
    model.train()
    total_loss = 0
    correct = 0 # number of correct preds (training)
    total = 0 # number of total samples (training)

    train_loop = tqdm(enumerate(train_loader), total=len(train_loader), desc=f"Epoch [{epoch+1}/{EPOCHS}]")

    for step, (videos, labels) in train_loop:
        videos, labels = videos.to(device), labels.to(device)

        outputs = model(videos)
        loss = criterion(outputs, labels)

        optimizer.zero_grad() # get gradients to zero
        loss.backward() # perform backward propogation
        optimizer.step() # optimize

        # Calculate loss
        total_loss += loss.item()
        _, preds = torch.max(outputs, 1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

        train_loop.set_postfix(loss=loss.item(), acc=correct / total)

    train_acc = correct / total
    print(f"Epoch [{epoch+1}/{EPOCHS}] Completed — Train Loss: {total_loss:.4f}, Train Accuracy: {train_acc:.4f}")

    # Validation phase
    model.eval()
    val_loss = 0
    val_correct = 0 # number of correct preds (validation)
    val_total = 0 # number of total samples (validation)

    with torch.no_grad():
        for videos, labels in val_loader:
            videos, labels = videos.to(device), labels.to(device)

            outputs = model(videos)
            loss = criterion(outputs, labels)
            val_loss += loss.item()

            _, preds = torch.max(outputs, 1)
            val_correct += (preds == labels).sum().item()
            val_total += labels.size(0)

    val_acc = val_correct / val_total
    print(f"Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_acc:.4f}")

    # Early stopping and checkpointing
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        epochs_no_improve = 0
        torch.save(model.state_dict(), model_path)
        print(f"Best model saved with validation accuracy: {best_val_acc:.4f}")
    else:
        epochs_no_improve += 1
        print(f"No improvement in validation accuracy for {epochs_no_improve} epoch(s)")

    if epochs_no_improve >= PATIENCE:
        print("Early stopping triggered due to no improvement in validation accuracy.")
        break


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 106MB/s]


🚀 Starting training from scratch.


Epoch [1/50]: 100%|██████████| 400/400 [1:14:31<00:00, 11.18s/it, acc=0.701, loss=0.528]


Epoch [1/50] Completed — Train Loss: 234.3320, Train Accuracy: 0.7006
Validation Loss: 21.0927, Validation Accuracy: 0.8750
Best model saved with validation accuracy: 0.8750


Epoch [2/50]: 100%|██████████| 400/400 [52:06<00:00,  7.82s/it, acc=0.783, loss=0.502]


...

Epoch [5/50]: 100%|██████████| 400/400 [51:46<00:00,  7.77s/it, acc=0.82, loss=0.95]


Epoch [5/50] Completed — Train Loss: 165.8595, Train Accuracy: 0.8200
Validation Loss: 13.8668, Validation Accuracy: 0.8750
No improvement in validation accuracy for 1 epoch(s)


Epoch [6/50]: 100%|██████████| 400/400 [51:53<00:00,  7.78s/it, acc=0.819, loss=0.742]


Epoch [6/50] Completed — Train Loss: 164.1229, Train Accuracy: 0.8187
Validation Loss: 12.5272, Validation Accuracy: 0.8850
No improvement in validation accuracy for 2 epoch(s)


Epoch [7/50]:  90%|█████████ | 362/400 [46:59<04:55,  7.77s/it, acc=0.832, loss=0.569]

## Save Model

In [ ]:
torch.save(model.state_dict(), "models/best_model_clstm.pth")

# Evaluation

In [ ]:
# Device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Model setup
model = CNNLSTMModel().to(device)
# model = CNNVideoClassifier().to(device)
# model = C3DModel().to(device)
model.load_state_dict(torch.load("models/best_model_clstm.pth"))
model.eval()

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 277MB/s]


CNNLSTMModel(
  (cnn): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (4): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True

In [ ]:
# Evaluate on test set
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for videos, labels in test_loader:
        videos, labels = videos.to(device), labels.to(device)
        outputs = model(videos)
        _, preds = torch.max(outputs, 1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

test_acc = correct / total
print(f"Test Accuracy: {test_acc:.4f}")

# Test on Videos

In [ ]:
# -------------------- Install dependencies --------------------
# !pip install torchvision --quiet
# !pip install matplotlib --quiet
from google.colab import files

In [ ]:
# ============ Load model ============= #

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = CNNLSTMModel()
model.load_state_dict(torch.load("models/best_model_clstm.pth", map_location=device))
model.to(device).eval()

  warnings.warn(

  warnings.warn(msg)



CNNLSTMModel(
  (cnn): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (4): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True

In [ ]:
uploaded = files.upload()  # Upload your .mp4 video
video_path = list(uploaded.keys())[0]
print("Uploaded:", video_path)

Saving NV_combined_video.mp4 to NV_combined_video (1).mp4
Uploaded: NV_combined_video (1).mp4


In [ ]:
## ============= Extract Frames ============= #

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])

def extract_frames(video_path, num_frames=16):
    """
      Extract Video Frames
    """
    cap = cv2.VideoCapture(video_path)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    frame_idxs = np.linspace(0, total_frames - 1, num=num_frames, dtype=int)
    frames = []
    idx_set = set(frame_idxs.tolist())
    count = 0

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        if count in idx_set:
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            frame = transform(Image.fromarray(frame))
            frames.append(frame)
        count += 1

    cap.release()

    if len(frames) == 0:
        raise ValueError("No frames extracted.")
    while len(frames) < num_frames:
        frames.append(frames[-1])  # Pad with last frame

    video_tensor = torch.stack(frames)  # [T, C, H, W]
    return video_tensor.unsqueeze(0)  # [1, T, C, H, W]


In [ ]:
# ============= Predict ============ #

with torch.no_grad():
    video_tensor = extract_frames(video_path).to(device)
    output = model(video_tensor)
    _, predicted = torch.max(output, 1)
    label = "🟥 Violent" if predicted.item() == 0 else "🟩 Non-Violent"
    print(f"Prediction: {label}")


Prediction: 🟩 Non-Violent


# Streamlit UI

In [ ]:
# Clear Streamlit Cache
import streamlit as st
st.cache_resource.clear()

### installations

In [ ]:
!pip install -q streamlit

In [ ]:
!pip install streamlit pyngrok -q

In [ ]:
!pip install streamlit pyngrok ffmpeg-python

In [ ]:
# Set ngrok authentication token
!ngrok config add-authtoken 2ycNlcF5rJDLaQ2EZTuU2ejmgzV_7m2fdgUujhzKoHtFRxJwc

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


### writing files

In [ ]:
%%writefile app.py
import streamlit as st
import torch
import torchvision.transforms as transforms
from PIL import Image
import cv2
import numpy as np
import tempfile
import os
from model import CNNLSTMModel  # Replace if you're switching models

# ---------- Page Config ----------
st.set_page_config(page_title="Violence Detection AI", page_icon="🚨", layout="wide")

# ---------- Custom CSS ----------
st.markdown("""
    <style>
    .title {
        font-size: 2.5em;
        font-weight: bold;
        text-align: center;
        color: #D7263D;
        margin-bottom: 5px;
    }
    .subtitle {
        text-align: center;
        font-size: 1.1em;
        margin-bottom: 30px;
        color: #444;
    }
    .box {
        border: 1px solid #ddd;
        padding: 20px;
        border-radius: 10px;
        background-color: white;
        box-shadow: 2px 2px 6px rgba(0,0,0,0.05);
        margin-top: 10px;
    }
    </style>
""", unsafe_allow_html=True)

# ---------- Sidebar ----------
# st.sidebar.image("https://cdn-icons-png.flaticon.com/512/2409/2409342.png" width='100')
st.sidebar.markdown("""
    <div style='text-align: center;'>
        <img src='https://cdn-icons-png.flaticon.com/512/2409/2409342.png' width='100'/>
    </div>
""", unsafe_allow_html=True)

st.sidebar.header("🔬 Project Overview")
st.sidebar.markdown("""
**Model Architectures:**
- CNN + LSTM (final model)
- Simple ResNet18
- 3D-CNN (video-level)

**Dataset:**
- Real-life Violence Dataset

**Goal:**
- Predict "Violent" or "Non-Violent" from video

**Made by:**
- Aroosh Ahmad
- Violence Detecction
""")

# ---------- Connect Links ----------
st.sidebar.markdown("### 🤝 Connect with Me")
st.sidebar.markdown("""
[![LinkedIn](https://img.shields.io/badge/-LinkedIn-blue?style=flat-square&logo=linkedin&logoColor=white)](https://www.linkedin.com/in/aroosh-ahmad/)
[![GitHub](https://img.shields.io/badge/-GitHub-black?style=flat-square&logo=github&logoColor=white)](https://github.com/arooshahmad)
[![Kaggle](https://img.shields.io/badge/-Kaggle-blue?style=flat-square&logo=kaggle&logoColor=white)](https://www.kaggle.com/arooshahmadds)
""")

model_state_dict_path = "/content/drive/MyDrive/violence_detection/models/best_model_clstm.pth"

# ---------- Load Model ----------
@st.cache_resource
def load_model():
    model = CNNLSTMModel()
    model.load_state_dict(torch.load(model_state_dict_path, map_location="cpu"))
    model.eval()
    return model

model = load_model()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# ---------- Transform ----------
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])

def extract_frames(video_path, num_frames=16):
    cap = cv2.VideoCapture(video_path)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    frame_idxs = np.linspace(0, total_frames - 1, num=num_frames, dtype=int)
    frames = []

    idx_set = set(frame_idxs.tolist())
    count = 0

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        if count in idx_set:
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            frame = transform(Image.fromarray(frame))
            frames.append(frame)
        count += 1

    cap.release()

    if len(frames) == 0:
        raise ValueError("No frames extracted from video.")

    while len(frames) < num_frames:
        frames.append(frames[-1])

    video_tensor = torch.stack(frames)  # [T, C, H, W]
    video_tensor = video_tensor.unsqueeze(0)  # [1, T, C, H, W]
    return video_tensor


# ---------- App Title ----------
st.markdown('<div class="title">🎥 Real-Time Violence Detection</div>', unsafe_allow_html=True)
st.markdown('<div class="subtitle">Upload a short video and detect violence using AI-powered deep learning models.</div>', unsafe_allow_html=True)

# ---------- Upload Form ----------
with st.container():
    with st.form(key="input_form"):
        st.markdown('<div class="box">', unsafe_allow_html=True)
        uploaded_video = st.file_uploader("📤 Upload a Video File", type=["mp4", "mov", "avi"])
        submit_btn = st.form_submit_button("🔍 Analyze Video")
        st.markdown("</div>", unsafe_allow_html=True)

# ---------- Prediction ----------
if uploaded_video:
    st.video(uploaded_video)
    with tempfile.NamedTemporaryFile(delete=False, suffix='.mp4') as tmp_file:
        tmp_file.write(uploaded_video.read())
        tmp_path = tmp_file.name

    if submit_btn:
        with st.spinner("⏳ Processing..."):
            video_tensor = extract_frames(tmp_path).to(device)
            with torch.no_grad():
                output = model(video_tensor)
                _, predicted = torch.max(output, 1)
                label = "🟥 Violent" if predicted.item() == 0 else "🟩 Non-Violent"

        st.markdown(f"""<div class="box" style="text-align: center; font-size: 1.3em;">
            🔎 <strong>Prediction:</strong> {label}
        </div>""", unsafe_allow_html=True)

# ---------- Expander ----------
with st.expander("📊 Architecture Comparison & Details"):
    st.markdown("""
| Model           | Description                                             | Pros                        | Accuracy |
|----------------|---------------------------------------------------------|-----------------------------|----------|
| **CNN + LSTM** | ResNet18 for spatial + LSTM for temporal features       | Best temporal reasoning     | ⭐ **92%** |
| Simple ResNet  | ResNet18 classifier without sequence modeling           | Lightweight, fast           | ~78%     |
| **3D CNN**     | 3D convolutions across time dimension                   | High video context capture  | ~85%     |

- **Frame Count**: 16 evenly sampled
- **Input Size**: 224x224 (resized)
- **Prediction**: Binary (Violent / Non-Violent)

🔗 **Dataset**: [Real-life Violence Dataset on Kaggle](https://www.kaggle.com/datasets/karandeep98/real-life-violence-and-nonviolence-data)

> 🚧 This system is a research prototype and not meant for production surveillance yet.
    """)

# ---------- Footer ----------
st.markdown("""
---
<center>
Made with ❤️ by <b>Aroosh Ahmad</b> • Violence Detection Project • 2025<br><br>

<a href="https://www.linkedin.com/in/arooshahmad-data/" target="_blank">
<img src="https://img.shields.io/badge/LinkedIn-blue?logo=linkedin&style=flat-square" height="25">
</a>
<a href="https://github.com/arooshahmad-data" target="_blank">
<img src="https://img.shields.io/badge/GitHub-181717?logo=github&style=flat-square" height="25">
</a>
<a href="https://www.kaggle.com/arooshahmadds" target="_blank">
<img src="https://img.shields.io/badge/Kaggle-20BEFF?logo=kaggle&style=flat-square" height="25">
</a>
</center>
""", unsafe_allow_html=True)


Overwriting app.py


In [ ]:
%%writefile model.py
import torch.nn as nn
import torchvision.models as models

class CNNLSTMModel(nn.Module):
    def __init__(self, hidden_dim=256, lstm_layers=1, num_classes=2):
        super(CNNLSTMModel, self).__init__()

        # Pretrained CNN (ResNet18)
        resnet = models.resnet18(pretrained=True)
        self.cnn = nn.Sequential(*list(resnet.children())[:-1])  # Remove FC layer
        self.feature_dim = 512  # ResNet18 final feature size

        # Freeze CNN (optional)
        for param in self.cnn.parameters():
            param.requires_grad = False

        # LSTM
        self.lstm = nn.LSTM(input_size=self.feature_dim, hidden_size=hidden_dim,
                            num_layers=lstm_layers, batch_first=True)

        # Classifier
        self.classifier = nn.Linear(hidden_dim, num_classes)

    def forward(self, x):  # x: [B, T, C, H, W]
        B, T, C, H, W = x.shape
        x = x.view(B * T, C, H, W)

        features = self.cnn(x)  # Output: (B*T, 512, 1, 1)

        features = features.view(B, T, self.feature_dim)  # Reshape to (B, T, 512)
        lstm_out, _ = self.lstm(features)  # (B, T, hidden_dim)
        final_output = lstm_out[:, -1, :]  # Take last timestep
        out = self.classifier(final_output)  # (B, num_classes)
        return out



Overwriting model.py


### Running app

In [ ]:
# Change Working Directory
%cd /content/drive/MyDrive/violence_detection

In [ ]:
# Clear Streamlit Cache
import streamlit as st
st.cache_data.clear()
st.cache_resource.clear()

2025-06-17 08:07:16.719 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager


**Note:** kill the streamlit process and ngrok tunnel in case cache does not get's cleared.

In [ ]:
# kill all previous sessions
!pkill streamlit
!pkill -f streamlit
!pkill -f ngrok

!rm -rf ~/.streamlit/

In [ ]:
from pyngrok import ngrok
import time

# Kill previous tunnels
ngrok.kill()

# Wait to avoid race condition
# time.sleep(3)

# Start ngrok with correct syntax
public_url = ngrok.connect(addr="8501", proto="http")
print(f"🔗 Streamlit App is Live: {public_url}")

# Run streamlit in background
!streamlit run app.py &>/content/logs.txt &

🔗 Streamlit App is Live: NgrokTunnel: "https://f999-34-148-30-96.ngrok-free.app" -> "http://localhost:8501"


# Misc Collect Video

In [ ]:
import cv2
import os
from google.colab import files
from natsort import natsorted
from glob import glob

def create_video_from_frames(video_id, folder_path, fps=25, violence = True):
    """
      Combines frames of video back to video format (.mp4) at 25 fps.
    """
    if violence:
      pattern = os.path.join(folder_path, f"V_{video_id}.mp4_frame*.jpg")
    else:
      pattern = os.path.join(folder_path, f"NV_{video_id}.mp4_frame*.jpg")


    frame_files = natsorted(glob(pattern))

    if not frame_files:
        print(f"❌ No frames found for video ID: {video_id}")
        return

    # Read the first frame to get dimensions
    first_frame = cv2.imread(frame_files[0])
    height, width, layers = first_frame.shape

    if violence:
      output_name = f"V_{video_id}.mp4"
    else:
      output_name = f"NV_{video_id}.mp4"


    out = cv2.VideoWriter(output_name, cv2.VideoWriter_fourcc(*'mp4v'), fps, (width, height))

    print(f"🔄 Generating video for ID {video_id} with {len(frame_files)} frames...")

    for frame_file in frame_files:
        frame = cv2.imread(frame_file)
        out.write(frame)

    out.release()
    print(f"✅ Video created: {output_name}")

    files.download(output_name)

# ---------- 🔧 User Configuration ----------

# Path to folder containing all frame images
folder_path = "/content/drive/MyDrive/violence_detection/images"  # Update if frames are elsewhere

# List of video IDs to process (just numbers, not filenames)
video_ids = [1000, 223, 512, 645, 798, 919]  # Add as many IDs as needed

# ---------- 🎬 Run Conversion ----------
for vid in video_ids:
    create_video_from_frames(vid, folder_path, violence=False)


🔄 Generating video for ID 1000 with 5 frames...
✅ Video created: NV_1000.mp4


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

🔄 Generating video for ID 223 with 5 frames...
✅ Video created: NV_223.mp4


<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

🔄 Generating video for ID 919 with 5 frames...
✅ Video created: NV_919.mp4


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
## Combining small chunks from videos

from moviepy.editor import VideoFileClip, concatenate_videoclips
import os
import shutil
from IPython.display import Javascript

# --- User Inputs ---
folder_path = "/content/drive/MyDrive/violence_detection/videos"  # ✅ CHANGE this to your Drive folder path
prefix = "NV_"
output_file = "NV_combined_video.mp4"
save_to_drive = True  # Set to False to trigger browser download instead

# --- Collect matching video files ---
video_files = sorted([
    os.path.join(folder_path, f)
    for f in os.listdir(folder_path)
    if f.startswith(prefix) and f.endswith((".mp4", ".mov", ".avi"))
])

if not video_files:
    print("❌ No matching videos found.")
else:
    print(f"🎬 Found {len(video_files)} video(s). Combining...")

    # --- Load and concatenate ---
    clips = [VideoFileClip(f) for f in video_files]
    final_clip = concatenate_videoclips(clips)

    # --- Save temp output ---
    temp_path = f"/content/{output_file}"
    final_clip.write_videofile(temp_path, codec="libx264", audio_codec="aac")

    if save_to_drive:
        # --- Save to same Drive folder ---
        dest_path = os.path.join(folder_path, output_file)
        shutil.move(temp_path, dest_path)
        print(f"✅ Saved to Google Drive: {dest_path}")
    else:
        # --- Trigger browser download ---
        print("⬇️ Downloading to browser...")
        display(Javascript(f'''
            var link = document.createElement('a');
            link.href = "{temp_path}";
            link.download = "{output_file}";
            document.body.appendChild(link);
            link.click();
            document.body.removeChild(link);
        '''))


In [ ]:
## Create Combined Video
import cv2
import os
import numpy as np
from glob import glob
from natsort import natsorted
from tqdm import tqdm

# --- Configuration ---
folder_path = "/content/drive/MyDrive/MSAI/real-life-violence-and-nonviolence-data/frames"  # Your frames folder
prefix = "NV_"  # Video ID/prefix like "NV_"
output_path = "/content/NV_combined_framewise.mp4"
fps = 5  # You can change this

# --- Step 1: Group frames by video ---
frame_dict = {}

for path in glob(os.path.join(folder_path, f"{prefix}*.jpg")):
    basename = os.path.basename(path)
    parts = basename.replace(".jpg", "").split("_")  # Example: NV_1_03 → ['NV', '1', '03']

    if len(parts) < 3:
        continue  # Skip invalid names

    video_id = f"{parts[0]}_{parts[1]}"  # e.g., NV_1
    frame_idx = parts[2]

    if frame_idx not in frame_dict:
        frame_dict[frame_idx] = []

    frame_dict[frame_idx].append((video_id, path))

# --- Step 2: Sort frame indexes ---
frame_keys = natsorted(frame_dict.keys())

# --- Step 3: Stack frames and write to video ---
video_writer = None
frame_size = None

for frame_idx in tqdm(frame_keys, desc="Processing frames"):
    frame_paths = natsorted([p[1] for p in frame_dict[frame_idx]])

    loaded_frames = [cv2.imread(fp) for fp in frame_paths]
    loaded_frames = [cv2.resize(f, (224, 224)) for f in loaded_frames if f is not None]

    if not loaded_frames:
        continue

    # Combine horizontally
    combined_frame = np.hstack(loaded_frames)

    # Initialize video writer
    if video_writer is None:
        height, width = combined_frame.shape[:2]
        frame_size = (width, height)
        fourcc = cv2.VideoWriter_fourcc(*"mp4v")
        video_writer = cv2.VideoWriter(output_path, fourcc, fps, frame_size)

    video_writer.write(combined_frame)

# --- Release Writer ---
if video_writer:
    video_writer.release()
    print(f"✅ Combined video saved at: {output_path}")
else:
    print("❌ No frames found or video not created.")


# Performance Summary

| Model                | Spatial Features | Temporal Modeling | Train Acc | Val Acc | Test Acc | Remarks                                         |
|---------------------|------------------|-------------------|-----------|---------|----------|--------------------------------------------------|
| **CNNVideoClassifier** | ✅ ResNet18       | ❌ None             | ~82.0%     | ~89.0%   |  ~78      | Fast & light, lacks temporal context             |
| **CNN + LSTM**         | ✅ ResNet18       | ✅ LSTM             | ~95%      | ~94%    | ~92%     | Best for temporal data (e.g. violence detection) |
| **3D CNN**             | ✅ 3D ConvNet     | ✅ Implicit         | ~87%      | ~85%    | -     | Captures spatiotemporal patterns directly        |

---

